In [6]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender
from MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender

#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [8]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')
UCM_all = scipy.sparse.load_npz('./Matrix/UCM_all_matrix.npz')



#Load playlistsIDS
data_playlists= pd.read_csv('all/train.csv',low_memory = False)
userID=data_playlists.playlist_id.unique()

In [3]:
CFrecommender = ItemKNNCFRecommender(URM_all)
CFrecommender.fit()

SLIMrecommender = SLIM_BPR_Cython(URM_all)
SLIMrecommender.fit()

CBFrecommender = ItemKNNCBFRecommender(ICM_all,URM_all)
CBFrecommender.fit()

SLIMErecommender = SLIMElasticNetRecommender(URM_all)
SLIMErecommender.fit()

UCFRecommender = UserKNNCFRecommender(URM_all)
UCFRecommender.fit()




Similarity column 20635 ( 100 % ), 2119.82 column/sec, elapsed time 0.16 min
SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Processed 1211791 ( 100.00% ) in 7.99 seconds. BPR loss is 1.06E+00. Sample per second: 151572
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.39 min
Processed 1211791 ( 100.00% ) in 7.88 seconds. BPR loss is 2.38E+00. Sample per second: 153871
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.51 min
Processed 1211791 ( 100.00% ) in 7.96 seconds. BPR loss is 3.29E+00. Sample per second: 152223
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.63 min
Processed 1211791 ( 100.00% ) in 7.90 seconds. BPR loss is 4.05E+00. Sample per second: 153399
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.74 min
Processed 1211791 ( 100.00% ) in 7.81 seconds. BPR loss is 4.67E+00. Sample per second: 155129
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.86 min
Processed 1211791 ( 100.00% ) in 8.03 seconds. BPR los

SLIM_BPR_Recommender: Epoch 52 of 100. Elapsed time 7.77 min
Processed 1211791 ( 100.00% ) in 8.34 seconds. BPR loss is 1.58E+01. Sample per second: 145326
SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 7.90 min
Processed 1211791 ( 100.00% ) in 8.23 seconds. BPR loss is 1.59E+01. Sample per second: 147205
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 8.03 min
Processed 1211791 ( 100.00% ) in 8.28 seconds. BPR loss is 1.61E+01. Sample per second: 146310
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 8.17 min
Processed 1211791 ( 100.00% ) in 8.81 seconds. BPR loss is 1.62E+01. Sample per second: 137506
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 8.31 min
Processed 1211791 ( 100.00% ) in 9.18 seconds. BPR loss is 1.63E+01. Sample per second: 131938
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 8.45 min
Processed 1211791 ( 100.00% ) in 7.77 seconds. BPR loss is 1.64E+01. Sample per second: 155940
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 8.57 min
Pro

In [ ]:
recommender_list = [
    #Random,
    #TopPop,
    #P3alphaRecommender,
    #RP3betaRecommender,
    #ItemKNNCFRecommender,
    #UserKNNCFRecommender,
    #MatrixFactorization_BPR_Cython,
    #MatrixFactorization_FunkSVD_Cython,
    #PureSVDRecommender,
    #SLIM_BPR_Cython,
    SLIMElasticNetRecommender
    ]


from Base.Evaluation.Evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(URM_test, [10], exclude_seen=True)


output_root_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_root_path):
    os.makedirs(output_root_path)


logFile = open(output_root_path + "result_all_algorithms.txt", "a")


for recommender_class in recommender_list:

    try:

        print("Algorithm: {}".format(recommender_class))


        recommender = recommender_class(URM_train)
        recommender.fit()
        result=recommender.evaluateRecommendations(URM_test)
        print("Recommender MAP is= {}".format(result["MAP"]))
        #results_run, results_run_string = evaluator.evaluateRecommender(recommender)
       
        #print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_run_string))
        #logFile.write("Algorithm: {}, results: \n{}\n".format(recommender.__class__, results_run_string))
        #logFile.flush()
 
        
    except Exception as e:
        traceback.print_exc()
        logFile.write("Algorithm: {} - Exception: {}\n".format(recommender_class, str(e)))
        logFile.flush()

Algorithm: <class 'SLIM_ElasticNet.SLIMElasticNetRecommender.SLIMElasticNetRecommender'>


#PER ESEGUIRE SUBMISSION
from createSubmission import create_submission

recommender=UserKNNCFRecommender(URM_all)
recommender.fit()

create_submission(recommender)

import re

results_run_string
m = re.search('MAP: (.+?),', results_run_string)
if m:
    found = m.group(1)
found

#PER CONTROLLARI DOPPIONI RACCOMANDAZIONE
racommandation = CFrecommender.recommend(userToPredict)
racommandation

target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
target_playlistIds=target_playlist.playlist_id.values
target_playlistIds.sort()
songs=data_playlists.loc[data_playlists['playlist_id'].isin(target_playlistIds)]
songs

counter=0
for x in target_playlistIds:
        raccomandate=racommandation[counter]
        dfsongs = songs.loc[songs['playlist_id'] == x]
        for row in dfsongs.itertuples():
            if row.track_id in raccomandate:
                print("ERRORRRR")
                print(x)
                print(row.track_id)
        counter=counter+1



In [4]:
from KNN.HybridScoresProva import HybridScoresProva

SuperSimilarity = ItemKNNSimilarityHybridRecommender(URM_all, SLIMrecommender, SLIMErecommender)
SuperSimilarity.fit(alpha=0.5, topK=600)
HSimilaritySuper = ItemKNNSimilarityHybridRecommender3(URM_all, SuperSimilarity, CFrecommender,CBFrecommender)
HSimilaritySuper.fit(topK=800, alpha=0.7, beta=0.4, gamma=0.2)



In [10]:
UserCBFrecommender = UserKNNCBFRecommender(UCM_all,URM_all) 
UserCBFrecommender.fit(topK=100, shrink=30,similarity='cosine',feature_weighting = "BM25")
SVDrecommender = PureSVDRecommender (URM_all)
SVDrecommender.fit(num_factors=1000,n_iter=40)

Hrecommender=ItemKNNScoresHybridRecommender(URM_all,UserCBFrecommender,SVDrecommender)
Hrecommender.fit(0.8)


Similarity column 28642 ( 57 % ), 920.39 column/sec, elapsed time 0.52 min
Similarity column 50446 ( 100 % ), 911.82 column/sec, elapsed time 0.92 min
PureSVDRecommender Computing SVD decomposition...
PureSVDRecommender Computing SVD decomposition... Done!


In [19]:
#creazione liste per rec con liste
train_df= pd.read_csv('all/train.csv',low_memory = False)
targetP_df= pd.read_csv('all/target_playlists.csv',low_memory = False)
targetP_df= targetP_df.playlist_id.values
#media delle interazione 23, min=7, max=75
resultDf = pd.DataFrame()
URM_all=URM_all.todense()

for x in targetP_df:
    c = np.count_nonzero(URM_all[x,])
    resultDf.at[x, "interactions"]=c

In [26]:
a= resultDf.loc[resultDf['interactions']< 8]
list_low_interaction_indexes=a.index.values
print(list_low_interaction_indexes.shape)

(95,)


In [25]:
from createSubmission import create_submission

Hrec=ItemKNNScoresHybridRecommender(URM_all,UserCBFrecommender,SVDrecommender)
Hrecommender.fit(0.8)

create_submission(Hrec)

In [ ]:
from createSubmission import create_submission


SuperSlimSimilarity=ItemKNNSimilarityHybridRecommender(URM_all,SLIMrecommender,SLIMErecommender,"l2")
SuperSlimSimilarity.fit(alpha=0.5, topK=600)

recommender=ItemKNNSimilarityHybridRecommender3(URM_all,SuperSlimSimilarity, CFrecommender,CBFrecommender,"l2")
recommender.fit(topK=600,alpha=0.6,beta=0.2,gamma=0.2)

create_submission(recommender)